In [229]:
# Notebook to inspect and clean the fixed SCL data Renee pulled

%matplotlib inline
import numpy as np
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
# from datetime import datetime
from datetime import date
import holidays
from dateutil import tz
import pytz

In [230]:
# Read file
SCL=pd.read_csv('SEA_DATA/Energy Consumption/ad_scl_II.csv', skiprows=0)

# Make adjustments to bring datetime into proper format
times = pd.to_datetime(SCL.Date, format="%m/%d/%Y")
SCL = SCL.drop(['Date','PST'], axis=1)
SCL.insert(0, 'date', times, False)
SCL.rename(columns={'Hour':'local_hour'}, inplace=True)
SCL['local_hour'].replace(24,0, inplace=True)

# Corrects an error with hour 24 not becoming new day
offset = pd.DateOffset(days=1)
SCL.loc[SCL.local_hour == 0, 'date'] = (SCL['date'] + offset)

# This information in the original sheet is incorrect
SCL.drop('Day of Week', axis=1, inplace=True)
SCL.drop('Day Name', axis=1, inplace=True)
SCL.drop('Month Name', axis=1, inplace=True)

# Let's fix that
SCL["day_of_week"]=SCL.date.dt.dayofweek
SCL["day_name"]=SCL.date.dt.weekday_name

SCL.head()

,date,local_hour,Issue,DF,D,AD,day_of_week,day_name
65,2015-07-03,18,1,1280.0,1236.0,1236,4,Friday
66,2015-07-03,19,1,1255.0,1204.0,1204,4,Friday
67,2015-07-03,20,1,1230.0,1171.0,1171,4,Friday
68,2015-07-03,21,1,1235.0,1134.0,1134,4,Friday
69,2015-07-03,22,1,1200.0,1128.0,1128,4,Friday
70,2015-07-03,23,1,1145.0,1096.0,1096,4,Friday
71,2015-07-04,0,1,1070.0,1026.0,1026,5,Saturday
72,2015-07-04,1,1,960.0,960.0,960,5,Saturday
73,2015-07-04,2,1,905.0,905.0,905,5,Saturday
74,2015-07-04,3,1,870.0,868.0,868,5,Saturday


In [231]:
# Add weekend check
SCL['weekend']=((SCL.date.dt.dayofweek // 5 == 1)).astype(int)

# Add holiday check
# If a federal holiday falls on Saturday it is usually observed on Friday (the day before),
# and when it falls on Sunday it is observed the following Monday. 
cal = calendar();
us_holidays=cal.holidays(start=SCL.date.min(), end=SCL.date.max())
SCL['holiday'] = SCL['date'].dt.date.astype('datetime64').isin(us_holidays)

# Add 3 day holiday weekend check
thday=[False]*SCL.shape[0]
for i in range(0,SCL['day_of_week'].shape[0]):
    if SCL['holiday'].iloc[i]==True:
        # on a Monday
        if SCL['day_of_week'].iloc[i]==0:
            thday[i]=True
            for j in range(0,49):
                thday[i-j]=True
        # on a Friday
        if SCL['day_of_week'].iloc[i]==4:
            thday[i]=True
            for j in range(0,49):
                thday[i+j]=True

SCL['3-day_holiday_wkend']=thday

In [232]:
SCL[4430:4490]

,date,local_hour,Issue,DF,D,AD,day_of_week,day_name,weekend,holiday,3-day_holiday_wkend
4430,2016-01-01,14,0,1365.0,1317.0,1317,4,Friday,0,True,True
4431,2016-01-01,15,0,1360.0,1291.0,1291,4,Friday,0,True,True
4432,2016-01-01,16,0,1390.0,1292.0,1292,4,Friday,0,True,True
4433,2016-01-01,17,1,1510.0,1367.0,1367,4,Friday,0,True,True
4434,2016-01-01,18,1,1640.0,1455.0,1455,4,Friday,0,True,True
4435,2016-01-01,19,1,1650.0,1450.0,1450,4,Friday,0,True,True
4436,2016-01-01,20,1,1600.0,1419.0,1419,4,Friday,0,True,True
4437,2016-01-01,21,1,1515.0,1392.0,1392,4,Friday,0,True,True
4438,2016-01-01,22,0,1430.0,1354.0,1354,4,Friday,0,True,True
4439,2016-01-01,23,1,1340.0,1293.0,1293,4,Friday,0,True,True


In [233]:
def lagged_values(df,name,lags):
    colIndex = df.columns.get_loc(name)
    for i in range(1,lags+1):
        label=name+"_"+str(i)
        nonIntChars = sum(c.isalpha() for c in label) + 1
        df.insert(colIndex+i, label, 0)
        for i in df.columns[df.columns.str.contains(name+"_")]:
            df[i] = df[name].shift(int(i[nonIntChars:]),fill_value = 0)

In [234]:
# Create 24hr lagged values of demand forecast, demand, and adjusted demand
lagged_values(SCL,'DF',24)
lagged_values(SCL,'D',24)
lagged_values(SCL,'AD',24)

In [235]:
# Create 24 hour average of all 3 values
SCL.insert(4, 'DF_LAST_24_AVG','')
SCL.insert(30, 'D_LAST_24_AVG','')
SCL.insert(56, 'AD_LAST_24_AVG','')

In [240]:
# I'm sure there's a nicer way to do this...
SCL['DF_LAST_24_AVG'] = round(SCL[['DF_1','DF_2','DF_3','DF_4','DF_5','DF_6','DF_7','DF_8','DF_9','DF_10','DF_11','DF_12','DF_13','DF_14','DF_15','DF_16','DF_17','DF_18','DF_19','DF_20','DF_21','DF_22','DF_23','DF_24']].mean(axis=1), 2)
SCL['D_LAST_24_AVG'] = round(SCL[['D_1','D_2','D_3','D_4','D_5','D_6','D_7','D_8','D_9','D_10','D_11','D_12','D_13','D_14','D_15','D_16','D_17','D_18','D_19','D_20','D_21','D_22','D_23','D_24']].mean(axis=1), 2)
SCL['AD_LAST_24_AVG'] = round(SCL[['AD_1','AD_2','AD_3','AD_4','AD_5','AD_6','AD_7','AD_8','AD_9','AD_10','AD_11','AD_12','AD_13','AD_14','AD_15','AD_16','AD_17','AD_18','AD_19','AD_20','AD_21','AD_22','AD_23','AD_24']].mean(axis=1), 2)

In [241]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
SCL[4430:4440]

,date,local_hour,Issue,DF,DF_LAST_24_AVG,DF_1,DF_2,DF_3,DF_4,DF_5,DF_6,DF_7,DF_8,DF_9,DF_10,DF_11,DF_12,DF_13,DF_14,DF_15,DF_16,DF_17,DF_18,DF_19,DF_20,DF_21,DF_22,DF_23,DF_24,D,D_LAST_24_AVG,D_1,D_2,D_3,D_4,D_5,D_6,D_7,D_8,D_9,D_10,D_11,D_12,D_13,D_14,D_15,D_16,D_17,D_18,D_19,D_20,D_21,D_22,D_23,D_24,AD,AD_LAST_24_AVG,AD_1,AD_2,AD_3,AD_4,AD_5,AD_6,AD_7,AD_8,AD_9,AD_10,AD_11,AD_12,AD_13,AD_14,AD_15,AD_16,AD_17,AD_18,AD_19,AD_20,AD_21,AD_22,AD_23,AD_24,day_of_week,day_name,weekend,holiday,3-day_holiday_wkend
4430,2016-01-01,14,0,1365.0,1391.25,1385.0,1415.0,1440.0,1435.0,1395.0,1300.0,1245.0,1180.0,1150.0,1140.0,1140.0,1150.0,1180.0,1295.0,1410.0,1520.0,1600.0,1635.0,1665.0,1665.0,1580.0,1485.0,1480.0,1500.0,1317.0,1341.38,1352.0,1382.0,1398.0,1386.0,1361.0,1336.0,1285.0,1212.0,1161.0,1143.0,1154.0,1177.0,1197.0,1240.0,1286.0,1358.0,1421.0,1484.0,1551.0,1588.0,1488.0,1411.0,1397.0,1425.0,1317,1341.38,1352,1382,1398,1386,1361,1336,1285,1212,1161,1143,1154,1177,1197,1240,1286,1358,1421,1484,1551,1588,1488,1411,1397,1425,4,Friday,0,True,True
4431,2016-01-01,15,0,1360.0,1385.62,1365.0,1385.0,1415.0,1440.0,1435.0,1395.0,1300.0,1245.0,1180.0,1150.0,1140.0,1140.0,1150.0,1180.0,1295.0,1410.0,1520.0,1600.0,1635.0,1665.0,1665.0,1580.0,1485.0,1480.0,1291.0,1336.88,1317.0,1352.0,1382.0,1398.0,1386.0,1361.0,1336.0,1285.0,1212.0,1161.0,1143.0,1154.0,1177.0,1197.0,1240.0,1286.0,1358.0,1421.0,1484.0,1551.0,1588.0,1488.0,1411.0,1397.0,1291,1336.88,1317,1352,1382,1398,1386,1361,1336,1285,1212,1161,1143,1154,1177,1197,1240,1286,1358,1421,1484,1551,1588,1488,1411,1397,4,Friday,0,True,True
4432,2016-01-01,16,0,1390.0,1380.62,1360.0,1365.0,1385.0,1415.0,1440.0,1435.0,1395.0,1300.0,1245.0,1180.0,1150.0,1140.0,1140.0,1150.0,1180.0,1295.0,1410.0,1520.0,1600.0,1635.0,1665.0,1665.0,1580.0,1485.0,1292.0,1332.46,1291.0,1317.0,1352.0,1382.0,1398.0,1386.0,1361.0,1336.0,1285.0,1212.0,1161.0,1143.0,1154.0,1177.0,1197.0,1240.0,1286.0,1358.0,1421.0,1484.0,1551.0,1588.0,1488.0,1411.0,1292,1332.46,1291,1317,1352,1382,1398,1386,1361,1336,1285,1212,1161,1143,1154,1177,1197,1240,1286,1358,1421,1484,1551,1588,1488,1411,4,Friday,0,True,True
4433,2016-01-01,17,1,1510.0,1376.67,1390.0,1360.0,1365.0,1385.0,1415.0,1440.0,1435.0,1395.0,1300.0,1245.0,1180.0,1150.0,1140.0,1140.0,1150.0,1180.0,1295.0,1410.0,1520.0,1600.0,1635.0,1665.0,1665.0,1580.0,1367.0,1327.50,1292.0,1291.0,1317.0,1352.0,1382.0,1398.0,1386.0,1361.0,1336.0,1285.0,1212.0,1161.0,1143.0,1154.0,1177.0,1197.0,1240.0,1286.0,1358.0,1421.0,1484.0,1551.0,1588.0,1488.0,1367,1327.50,1292,1291,1317,1352,1382,1398,1386,1361,1336,1285,1212,1161,1143,1154,1177,1197,1240,1286,1358,1421,1484,1551,1588,1488,4,Friday,0,True,True
4434,2016-01-01,18,1,1640.0,1373.75,1510.0,1390.0,1360.0,1365.0,1385.0,1415.0,1440.0,1435.0,1395.0,1300.0,1245.0,1180.0,1150.0,1140.0,1140.0,1150.0,1180.0,1295.0,1410.0,1520.0,1600.0,1635.0,1665.0,1665.0,1455.0,1322.46,1367.0,1292.0,1291.0,1317.0,1352.0,1382.0,1398.0,1386.0,1361.0,1336.0,1285.0,1212.0,1161.0,1143.0,1154.0,1177.0,1197.0,1240.0,1286.0,1358.0,1421.0,1484.0,1551.0,1588.0,1455,1322.46,1367,1292,1291,1317,1352,1382,1398,1386,1361,1336,1285,1212,1161,1143,1154,1177,1197,1240,1286,1358,1421,1484,1551,1588,4,Friday,0,True,True
4435,2016-01-01,19,1,1650.0,1372.71,1640.0,1510.0,1390.0,1360.0,1365.0,1385.0,1415.0,1440.0,1435.0,1395.0,1300.0,1245.0,1180.0,1150.0,1140.0,1140.0,1150.0,1180.0,1295.0,1410.0,1520.0,1600.0,1635.0,1665.0,1450.0,1316.92,1455.0,1367.0,1292.0,1291.0,1317.0,1352.0,1382.0,1398.0,1386.0,1361.0,1336.0,1285.0,1212.0,1161.0,1143.0,1154.0,1177.0,1197.0,1240.0,1286.0,1358.0,1421.0,1484.0,1551.0,1450,1316.92,1455,1367,1292,1291,1317,1352,1382,1398,1386,1361,1336,1285,1212,1161,1143,1154,1177,1197,1240,1286,1358,1421,1484,1551,4,Friday,0,True,True
4436,2016-01-01,20,1,1600.0,1372.08,1650.0,1640.0,1510.0,1390.0,1360.0,1365.0,1385.0,1415.0,1440.0,1435.0,1395.0,1300.0,1245.0,1180.0,1150.0,1140.0,1140.0,1150.0,1180.0,1295.0,1410.0,1520.0,1600.0,1635.0,1419.0,1312.71,1450.0,1455.0,1367.0,1